# Chemical Kinetics and Numerical Integration

Here we will use methods of numerical integration to solve for the abundances of the H$_3^+$ isotopologues in the ion trap experiment from last week's notebook. After using integrated rate equations and curve fitting, we came up with this result:

![Deuteration Results](https://leeping.github.io/che155/assets/images/week05/deuteration_results.png)

The deviations, most notable in the D$_2$H$^+$ results, are because the reverse reactions were not included in our model. It would be very difficult to derive new rate equations, so we will use numerical methods instead.

## Forward Euler Method

First, we will reimplement the exact same model as last time, but this time we will solve using the Forward Euler Method. First, load in the `deuteration.csv` file. It contains the same experimental data as last week, but the time field has been rounded and lined up so that all abundances for each molecule are given at the same time values. This will make comparisons with the numerical models easier down the road.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

df = pd.read_csv('deuteration.csv')
    
df

As a reminder, the model was defined by the equations:

$$ \frac{\text{d}[\text{H}_3^+]}{\text{d}t} = -k_1[\text{H}_3^+] $$

$$ \frac{\text{d}[\text{H}_2\text{D}^+]}{\text{d}t} = k_1[\text{H}_3^+] - k_2[\text{H}_2\text{D}^+] $$

$$ \frac{\text{d}[\text{D}_2\text{H}^+]}{\text{d}t} = k_2[\text{H}_2\text{D}^+] - k_3[\text{D}_2\text{H}^+] $$

$$ \frac{\text{d}[\text{H}_3^+]}{\text{d}t} = k_3[\text{D}_2\text{H}^+] $$

We can express these in a simple form with the matrix equation:

$$ \begin{bmatrix} \text{d}[\text{H}_3^+]/\text{d}t \\ \text{d}[\text{H}_2\text{D}^+]/\text{d}t \\ \text{d}[\text{D}_2\text{H}^+]/\text{d}t \\ \text{d}[\text{D}_3^+]/\text{d}t \end{bmatrix} = \begin{bmatrix} -k_1 & 0 & 0 & 0 \\ k_1 & -k_2 & 0 & 0 \\ 0 & k_2 & -k_3 & 0 \\ 0 & 0 & k_3 & 0 \end{bmatrix} \begin{bmatrix}[\text{H}_3^+] \\ [\text{H}_2\text{D}^+] \\ [\text{D}_2\text{H}^+] \\ [\text{D}_3^+] \end{bmatrix} $$

Then, taking a time step $\Delta t$, we can compute new concentrations:

$$ \begin{bmatrix}[\text{H}_3^+] \\ [\text{H}_2\text{D}^+] \\ [\text{D}_2\text{H}^+] \\ [\text{D}_3^+] \end{bmatrix}_{\,i+1} = \begin{bmatrix}[\text{H}_3^+] \\ [\text{H}_2\text{D}^+] \\ [\text{D}_2\text{H}^+] \\ [\text{D}_3^+] \end{bmatrix}_{\,i} +  \begin{bmatrix} -k_1 & 0 & 0 & 0 \\ k_1 & -k_2 & 0 & 0 \\ 0 & k_2 & -k_3 & 0 \\ 0 & 0 & k_3 & 0 \end{bmatrix} \begin{bmatrix}[\text{H}_3^+] \\ [\text{H}_2\text{D}^+] \\ [\text{D}_2\text{H}^+] \\ [\text{D}_3^+] \end{bmatrix}_{\,i} \Delta t$$

As of Python 3.5, matrix multiplication (and other types of dot products) can be done with the `@` operator. When used with `numpy.ndarray` objects, the [`numpy.matmul`](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html) function is called. In our case, we will create a 4x4 matrix called `J` and a 1D array with 4 elements called `n` to store the abundances. When we call `J@n`, it multiplies each row of `J` by the 4 elements in `n`, and adds them up. Here we use the results from the curve fitting to ideally give us similar results as last time. We will set the step size `dt` to 0.1 ms, and take 1500 steps.

In [ ]:
#iniitialize rate constants
hd=6.3e10
k1=1.42e-9*hd
k2=1.33e-9*hd
k3=1.05e-9*hd

#H3+ at t=0 is 932, H2D+, D2H+, and D3+ start at 0.
n0 = np.array([932,0,0,0])

#initialize an empty 4x4 matrix, and plug in k values at the right places
J = np.zeros((4,4))
J[0,0] = -k1
J[1,1] = -k2
J[2,2] = -k3
J[1,0] = k1
J[2,1] = k2
J[3,2] = k3

#this array n will be updated with the new concentrations at each step. Initialize it at n0
n = n0
dt = 1e-4
steps = 1500

#this array will keep track of the values of n at each step
nt = np.zeros((steps+1,len(n0)))
nt[0] = n0

#take each steps, updating n at each one; store the results in the nt array
for i in range(0,steps):
    n = n + J@n*dt
    nt[i+1] = n
        
nt

Now we can plot the results and compare with the experimental data.

In [ ]:
fig,ax = plt.subplots()
t = np.linspace(0,150e-3,len(nt))

ax.scatter(df['time'],df['H3+'],color='#000000',label=r'H$_3^+$')
ax.scatter(df['time'],df['H2D+'],color='#ffbf00',label=r'H$_2$D$^+$')
ax.scatter(df['time'],df['D2H+'],color='#022851',label=r'D$_2$H$^+$')
ax.scatter(df['time'],df['D3+'],color='#c10230',label=r'D$_3^+$')

ax.set_xlabel("Time (s)")
ax.set_ylabel("Number")

lines = ax.plot(t,nt)
lines[0].set_color('#000000')
lines[1].set_color('#ffbf00')
lines[2].set_color('#022851')
lines[3].set_color('#c10230')
ax.set_yscale('log')
ax.legend()

Note that the step size is a critical parameter! If we increase the step size too much, we can get some bad results.

In [ ]:
n = n0
dt = 5e-3
steps = round(.15/dt)+1

nt = np.zeros((steps+1,len(n0)))
nt[0] = n0

for i in range(0,steps):
    n = n + J@n*dt
    nt[i+1] = n
    
fig,ax = plt.subplots()
t = np.linspace(0,len(nt)*dt,len(nt))

ax.scatter(df['time'],df['H3+'],color='#000000',label=r'H$_3^+$')
ax.scatter(df['time'],df['H2D+'],color='#ffbf00',label=r'H$_2$D$^+$')
ax.scatter(df['time'],df['D2H+'],color='#022851',label=r'D$_2$H$^+$')
ax.scatter(df['time'],df['D3+'],color='#c10230',label=r'D$_3^+$')

ax.set_xlabel("Time (s)")
ax.set_ylabel("Number")

lines = ax.plot(t,nt)
lines[0].set_color('#000000')
lines[1].set_color('#ffbf00')
lines[2].set_color('#022851')
lines[3].set_color('#c10230')
ax.set_yscale('log')

## Least Squares Fitting and Numerical Integration

It is possible (though not very common) to implement least squares fitting together with the numerical integration in order to estimate the kinetics parameters. We'll walk through the process here. Last time we used `scipy.optimize.least_squares`, which required us to calculate the residuals vector between the model and the experimental data. When using integrated rate equations, this was straightforward because we could just plug in the time for each data point into the model and compute the model's prediction. With numerical integration; however, we do not have such a function!

Instead, what we can do is save the model's outputs whenever the time matches the time at which an experimental data point is taken. If we choose time steps judiciously, we can make sure that we always sample the model at each time point needed. If we inspect the data frame, we can see that all of the time points are at a multiple of 0.1 ms.

In [ ]:
df

Therefore, a time step `dt` of 0.1 ms (or some integer factor smaller) will ensure that the model samples each time point we need to compare with the experimental data. The code below checks to see if `i` (the current time in units of `dt`) is in the array `tvals`, which is the time array converted to units of dt, and if so it stores the current model abundances in a list for later use. Importantly, this is chosen such that all of the time comparisons are between integers so that we don't have to worry about issues with floating point comparisons.

At the end of the clode block, `nm` is a 2D numpy array where each row is a time point and each column is the abundance of one of the ions.

In [ ]:
n = n0
dt = 1e-4
steps = 1500

nmodel = []
tvals = df['time'].to_numpy()/dt
tvals = tvals.astype(int)

for i in range(0,steps+1):
    n = n + J@n*dt
    if i in tvals:
        nmodel.append(n)
        
nm = np.array(nmodel)
nm

Now we'll plot the results. A quick side note here: we've been doing a lot of repetitive manual color changing. If you have a set of colors you want to consistently use, you can change matplotlib's default color cycling (see this [tutorial](https://matplotlib.org/tutorials/intermediate/color_cycle.html) for a quick example). Below I create a new `cycler` object that tells matplotlib to cycle between the 4 colors we have been using instead of its defaults. As the tutorial shows, you can either set the cycler on an `Axes` object like in the code below, which only affects that object, or you can apply the cycler to all subsequently created plots.

In [ ]:
from cycler import cycler

ucd_cycler = (cycler(color=['#000000','#ffbf00','#022851','#c10230','#266041','#8a532f']))

fig,ax = plt.subplots()
ax.set_prop_cycle(ucd_cycler)
ax.plot(df['time'],nm,'o')

Now let's turn that into a function that takes the kinetics parameters (`h30`, `k1`, `k2`, `k3`) as arguments. We also need to pass the time values at which the model should be sampled, the step size, and the number of steps.

In [ ]:
def runmodel(params,tvals,dt,steps):
    h30 = params[0]
    k1 = params[1]
    k2 = params[2]
    k3 = params[3]
    n = np.asarray([h30,0,0,0])
    nmodel = []
    J = np.zeros((4,4))
    J[0,0] = -k1
    J[1,1] = -k2
    J[2,2] = -k3
    J[1,0] = k1
    J[2,1] = k2
    J[3,2] = k3

    for i in range(0,steps+1):
        n = n + J@n*dt
        if i in tvals:
            nmodel.append(n)
            
    return(np.array(nmodel))
    
    

Test to make sure the `runmodel` function works as intended:

In [ ]:
tvals = df['time'].to_numpy()/dt
tvals = tvals.astype(int)
hd=6.3e10
k1=1.43e-9*hd
k2=1.33e-9*hd
k3=1.05e-9*hd
h30 = 932

runmodel(np.array([h30,k1,k2,k3]),tvals,1e-4,1500)

To perform the `least_squares` optimization, we need to create a function that computes the residuals of the model. This function must have the signature `f(x,*args,**kwargs)` where `x` is an array containing the parameters that will be optimized (`h30`, `k1`, `k2`, and `k3`), `*args` contains any additional arguments that are needed, and `**kwargs` can contain any other information.

Like last time, we'll use `**kwargs` to pass in the experimental data. `*args` will contain the `tvals`, `dt`, and `steps` parameters that need to be passed to `runmodel.` Ance we have the results of the model, we need to compute the residuals.

In [ ]:
def total_fit(x,*args,**kwargs):
    
    df = kwargs['df']

    nm = runmodel(x,*args)
    
    #a naive algorithm using for loops; slow, but flexible!
#    out = []
#    for i,model in enumerate(nm):
#        for j,mol in enumerate(['H3+','H2D+','D2H+','D3+']):
#            n = df.at[i,mol]
#            if np.isfinite(n):
#                out.append(n-model[j])
#    return out
                
    #taking advantage of numpy's array routines: fast, but requires more work if anything changes
    rh3 = df['H3+'] - nm[:,0]
    rh3 = rh3[~np.isnan(rh3)] #remove NaNs... isnan returns an array of booleans, so we take the logical not and use it as a slice to extract only the finite values
    
    rh2d = df['H2D+'] - nm[:,1]
    rh2d = rh2d[~np.isnan(rh2d)]
    
    #there are no NaNs in the experimental data for D2H+ or D3+
    rd2h = df['D2H+'] - nm[:,2]
    rd3 = df['D3+'] - nm[:,3]
    
    #concatenate and return
    return np.concatenate((rh3,rh2d,rd2h,rd3))
        

Now we can use `least_squares` to compute optimal parameters, and we can see that we get almost exactly the same results as the integrated rate equation approach. Note, however, that there is no problem with us starting out with `k1` and `k2` being equal! There is no divide by 0 error with numerical integration like there was with the integrated rate equations.

In [ ]:
import scipy.optimize as opt
import numpy.linalg as la

data = {
    'df' : df
}

tvals = df['time'].to_numpy()/dt
tvals = tvals.astype(int)
hd=6.3e10

result = opt.least_squares(total_fit,[900,1e-9*hd,1e-9*hd,1e-9*hd],
                           args=[tvals,1e-4,1500],kwargs=data,verbose=1)
pcov = la.inv(result.jac.T @ result.jac)

for i,x in enumerate(['[H3+]0','k1','k2','k3']):
    den = hd
    if i==0:
        den = 1.
    print(f'{x} = {result.x[i]/den:.2e} +/- {np.sqrt(pcov[i][i])/den:.2e}')

## Integration with `scipy.integrate`

Our manual implementation of the numerical integration uned the Forward Euler Method, whose total error is proportional to $(\Delta t)^{1}$. It is usually desirable to use a higher-order method to achieve either higher accuracy or obtain the same accuracy with fewer steps. The function we are going to explore is [`scipy.integrate.solve_ivp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html), which is made to solve initial value problems.

In [ ]:
import scipy.integrate as spi

spi.solve_ivp?

As we can see from the function description, we need to provide at least 3 arguments:
- `fun` is a function that computes the vector of derivatives. Its function signature needs to be `f(t,y,*args)`. `t` is the current time, `y` is the current state array (in our case, the array containing the molecule abundances), and the remainder of the arguments can contain anything else needed to compute the derivatives (e.g., rate coefficients, etc)
- `t_span` is a tuple that specifies the initial and final time for the integration
- `y0` is a vector containing the initial conditions - the starting abundances for the molecules.

In addition to those required parameters, there are three other optional arguments that are useful for us:
- `method` selects which numerical integration method will be employed. The default, `'RK45'`, is the fourth-order Runge-Kutta method, but several others are available, including some implicit solvers that are important when problems are "stiff." A system of equations is stiff when the solutions are very sensitive to the step size even when the solution appears "smooth." Chemical kinetics problems are frequently stiff when there are some very slow reactions combined with others that are very fast, and you want to evaluate the system over a long time compared with the rate of the fast reactions. In the current example, all of the reactions have comparable rates, so we will stick with `'RK45'`, but often the `'Adams'` or `'Radau'` methods are more appropriate for kinetics problems.
- `t_eval` is a list of times at which the model returns abundances. If this is None, the model only gives the results at the final time. If we pass an array of times, the results will contain the abundances at all of the time values specified in `t_eval` which fall within `t_span`
- `dense_output` causes the solver to construct functions that interpolate between time steps. This allows you to (approximately) evaluate the model at any time, not just at the time steps that were used in the model.

Note that nowhere do you need to specify the step size! All of the methods employ various algorithms to automatically determine the step size needed to bring the error down to a certain desired value. Some even include adaptive step sizes that can take smaller or larger steps depending on the magnitudes of the derivatives.

Let's re-implement the same model, but this time perform the integration with `solve_ivp`. First we need to write a function that computes the derivative.

In [ ]:
# function must take t and y as its first 2 arguments. Since our derivatives don't explicitly depend on t, that variable isn't used in the body of the function.
# to calculate the rates, we need the rate coefficients and abundances. The abundances are in y, so we need to pass the k values as arguments.
def calc_derivative(t,y,k1,k2,k3):
    
    J = np.zeros((len(y),len(y)))
    J[0,0] = -k1
    J[1,1] = -k2
    J[2,2] = -k3
    J[1,0] = k1
    J[2,1] = k2
    J[3,2] = k3
    
    return J@y

With that, we can now use `solve_ivp` to compute the solution from 0 to 0.15 seconds. We'll use the default `RK45` integrator, and set the `dense_output` flag to allow us to generate a quasi-continuous model function. In addition, we'll pass our `df['time']` array to `t_eval` so that we have the exact model values at the experimental time points.

Within the `result` object that is returned, we can access the dense solution with `result.sol`, which takes a time value as an argument. The solution values are in `result.y`, and the time points for each solution are in `result.t`. The plot that this cell creates shows both the dense output and the discrete solutions.

In [ ]:
hd=6.3e10
k1=1.0e-9*hd
k2=1.0e-9*hd
k3=1.0e-9*hd
h30 = 932

result = spi.solve_ivp(calc_derivative,(0,.15),y0=[h30,0,0,0],
                       t_eval=df['time'],method='RK45',
                       dense_output=True,args=(k1,k2,k3))
fig,ax = plt.subplots()
ax.set_prop_cycle(ucd_cycler)
ax.scatter(df['time'],df['H3+'],color='#000000',label=r'H$_3^+$')
ax.scatter(df['time'],df['H2D+'],color='#ffbf00',label=r'H$_2$D$^+$')
ax.scatter(df['time'],df['D2H+'],color='#022851',label=r'D$_2$H$^+$')
ax.scatter(df['time'],df['D3+'],color='#c10230',label=r'D$_3^+$')
t = np.linspace(0,160e-3,1000)
ax.plot(t,result.sol(t).T)
#ax.plot(result.t,result.y.T,'o')
ax.set_yscale('log')

## Extending the System

We wish to add the reverse reactions to the system and also to explicitly model the reactions using the full second-order rates instead of the pseudo-first-order ones we have been using up until this points. So far our system has been explicitly hardcoded. This is fine for a small system like this, but if we start to have many more molecules and reactions, manually coding the rates is tedious and also error-prone.

We will aim to improve the reliability and flexibility of the code by defining the model in terms of chemical reactions, and we will automatically generate the rate equations from the reactions themselves. First, let's create a list of molecules. We'll use a pandas dataframe for convenience, though we could implement this with lists or numpy arrays as well.

We'll start by refactoring the existing pseudo-first-order system, and then show how we can easily convert to the full second order reaction network.

In [ ]:
species = pd.DataFrame(['H3+','H2D+','D2H+','D3+'],columns=['name'])
species

Now each molecule will be referred to by its index in this dataframe instead of by its name. We next need to define the chemical reactions that link these molecules together. We'll do this by creating a new class that contains the reactants, the products, and the rate coefficient. The class first extracts the unique reactants and products along with how many times that reactant/product appears, and stores those numbers as as numpy arrays, and uses  We also make a `__str__` function for convenience that will print the reaction and its rate. We pass the `species` data into the constructor so that the reaction can get the names of the molecules.

In [ ]:
class Reaction:
    def __init__(self,species,reactants=[],products=[],k=0.0):
        self.reactants, self.rcounts = np.unique(np.asarray(reactants),return_counts=True)
        self.products, self.pcounts  = np.unique(np.asarray(products),return_counts=True)
        rnames = []
        pnames = []
        for r,c in zip(self.reactants,self.rcounts):
            rnames.append(self.makename(species,c,r))
        for p,c in zip(self.products,self.pcounts):
            pnames.append(self.makename(species,c,p))
        self.k = k
        self.name = f'{" + ".join(rnames)} --> {" + ".join(pnames)}, k = {self.k:.2e}'
        
    def __str__(self):
        return self.name
    
    def makename(self,species,c,n):
        out = species.at[n,'name']
        if c > 1:
            out = f'{c}{out}'
        return out
        

To create a reaction, we call the `Reaction` constructor and give it the species list, then the list of the reactants' ID numbers, then the products' ID numbers, and then the rate coefficient. Since we're currently only considering the forward reactions and keeping \[HD\] constant, we can just include the ions.

In [ ]:
r1 = Reaction(species,[0],[1],1.43e-9*hd)
r2 = Reaction(species,[1],[2],1.33e-9*hd)
r3 = Reaction(species,[2],[3],1.05e-9*hd)
reactions = pd.DataFrame([r1,r2,r3],columns=['reaction'])
reactions

Note that we can make reactions that involve multiple of the same molecule. A silly example:

In [ ]:
print(Reaction(species,[0,0,1],[3,3,2],1.))

For computing the derivatives, we can use the definitions of the rate of an elementary reaction. For example, the elementary reaction A + B --> C + D has the following rates:

$$ -\frac{\text{d}[A]}{\text{d}t} = -\frac{\text{d}[B]}{\text{d}t} = \frac{\text{d}[C]}{\text{d}t} = \frac{\text{d}[D]}{\text{d}t} = k[\text{A}][\text{B}] $$

If the reaction has the form 2A --> C + D; this can also be written as A + A --> C + D, and the only difference is that the rate of change for \[A\] is twice as fast as the change for each product:

$$ -\frac{1}{2}\frac{\text{d}[A]}{\text{d}t} = \frac{\text{d}[C]}{\text{d}t} = \frac{\text{d}[D]}{\text{d}t} = k[\text{A}]^2 = k[\text{A}][\text{A}]  $$

What this means is that for each molecule, we just need to loop over the reactions, and each time the molecule appears as a reactant, we subtract its rate coefficient times the product of the reactants, and each time it appears as a product, we add k times the product of the reactants. This will work even if the molecule appears twice in the same reaction (either as a reactant or a product, or even both!), becase we'll add the rate once for each time the molecule appears in the reaction.

The code below is a new implementation of the derivative calculation that does this. It loops over the reactions, and for each reactant it subtracts the rate, and for each product it adds the rate.

In [ ]:
def calc_derivative_2(t,y,rxns):
    out = np.zeros_like(y)
    for r in rxns['reaction']:        
        out[r.reactants] -= r.k*np.prod(np.power(y[r.reactants],r.rcounts))*r.rcounts
        out[r.products] += r.k*np.prod(np.power(y[r.reactants],r.rcounts))*r.pcounts
        
    return out
        

This code takes advantage of numpy's advanced indexing capabilities. The lists of unique reactant and product IDs are used as indices to choose which concentrations to include in the rate calculations as well as which concentration derivatives to change. Note that the rates depend on the concentrations of the reactants, not the concentrations of the products. Below is some sample code showing how this works. The reaction is 3A + B --> 2C + D. Rate = k\[A\]^3\[B\] = (1e-4)(10)^3(3) = 0.3. So the concentration of A should change by -(3)(0.3) = -0.9, B should change by -0.3, C should change by +(2)(0.6) = 0.6, and D by +0.3

In [ ]:
y = np.asarray([10.,3.,7.,2.])
out = np.zeros_like(y)
reactants = np.asarray([0,1,0,0])
products = np.asarray([2,2,3])
reactants, rcounts = np.unique(reactants,return_counts=True)
products, pcounts = np.unique(products,return_counts=True)
out[reactants] += -1e-4*np.prod(np.power(y[reactants],rcounts))*rcounts
out[products] += 1e-4*np.prod(np.power(y[reactants],rcounts))*pcounts
out

And now, as a last sanity check, we should be able to plug in our reactions and initial conditions into the solver and get the same results.

In [ ]:
r1 = Reaction(species,[0],[1],1.43e-9*hd)
r2 = Reaction(species,[1],[2],1.33e-9*hd)
r3 = Reaction(species,[2],[3],1.05e-9*hd)
reactions = pd.DataFrame([r1,r2,r3],columns=['reaction'])
result = spi.solve_ivp(calc_derivative_2,(0,.16),y0=[932,0,0,0],t_eval=df['time'],method='RK45',dense_output=True,args=[reactions])
fig,ax = plt.subplots()
ax.set_prop_cycle(ucd_cycler)
t = np.linspace(0,160e-3,1000)
ax.plot(t,result.sol(t).T)
ax.plot(result.t,result.y.T,'o')

## Second Order Kinetics and Reverse Reactions

With our `Reaction` class and `calc_derivative_2` functions, it is now easy to include H2 and HD in the model, and do the second-order chemistry. The only addition is that we need to be careful about units. In the rate equations, the concentrations are given in molecules per cubic centimeter, so we need to divide the ion counts by the trap volume, which we do not exactly know. It may be listed in one of the many papers the group has published. However, the volume is likely on the order of 1 cubic centimeter. We can use that for now, and show that the final results in the end are not very sensitive to this number unless it's smaller than ~10-5, which seems physically impossible.

In [ ]:
species = pd.DataFrame(['H3+','H2D+',"D2H+","D3+",'HD','H2'],columns=['name'])

reactions = [Reaction(species,[0,4],[1,5],k=1.43e-9),
             Reaction(species,[1,4],[2,5],k=1.33e-9),
             Reaction(species,[2,4],[3,5],k=1.05e-9)]

reactions = pd.DataFrame(reactions,columns=['reaction'])
print(reactions)

volume = 1

result = spi.solve_ivp(calc_derivative_2,(0,.15),y0=[932/volume,0,0,0,6.3e10,0],t_eval=df['time'],method='RK45',dense_output=True,args=[reactions])
fig,ax = plt.subplots(figsize=(10,8))
ax.set_prop_cycle(ucd_cycler)
t = np.linspace(0,150e-3,1000)
lines = ax.plot(t,(result.sol(t)).T)
for l,n in zip(lines,species['name']):
    l.set_label(n)
ax.scatter(df['time'],df['H3+']/volume,color='#000000')
ax.scatter(df['time'],df['H2D+']/volume,color='#ffbf00')
ax.scatter(df['time'],df['D2H+']/volume,color='#022851')
ax.scatter(df['time'],df['D3+']/volume,color='#c10230')
ax.set_xlim(0,150e-3)
ax.set_ylabel('Abundance (cm$^{-3}$)')
ax.set_yscale('log')
ax.legend()

We can see from this graph why the pseudo-first-order approximation is so good: if there are only ~1000 ions in a cubic centimeter, there are over 6e10 HD molecules. Even after all 1000 H$_3^+$ ions are converted to D$_3^+$, only 3000 of the HD molecules disappeared, which is negligible. However, eventually if we make the trap volume small enough, we can start to see an effect on the model. For instance, here we make the trap volume 1.5e-3, which means there are roughly as many H$_3^+$ ions as HD molecules. The chemistry is qualitatively different, yet we did not have to rederive any rate equations. Numerical integration is versatile.

In [ ]:
species = pd.DataFrame(['H3+','H2D+',"D2H+","D3+",'HD','H2'],columns=['name'])

reactions = [Reaction(species,[0,4],[1,5],k=1.43e-9),
             Reaction(species,[1,4],[2,5],k=1.33e-9),
             Reaction(species,[2,4],[3,5],k=1.05e-9)]

reactions = pd.DataFrame(reactions,columns=['reaction'])
print(reactions)

volume = 1.5e-8

result = spi.solve_ivp(calc_derivative_2,(0,.15),y0=[932/volume,0,0,0,6.3e10,0],t_eval=df['time'],method='RK45',dense_output=True,args=[reactions])
fig,ax = plt.subplots(figsize=(10,8))
ax.set_prop_cycle(ucd_cycler)
t = np.linspace(0,150e-3,1000)
lines = ax.plot(t,(result.sol(t)).T)
for l,n in zip(lines,species['name']):
    l.set_label(n)
ax.scatter(df['time'],df['H3+']/volume,color='#000000')
ax.scatter(df['time'],df['H2D+']/volume,color='#ffbf00')
ax.scatter(df['time'],df['D2H+']/volume,color='#022851')
ax.scatter(df['time'],df['D3+']/volume,color='#c10230')
ax.set_xlim(0,150e-3)
ax.set_ylabel('Abundance (cm$^{-3}$)')
ax.set_yscale('log')
ax.legend()

Returning to more reasonable volumes, we can turn on the reverse reactions and see what happens. The paper says that the reverse reactions occur with rate coefficients that are of order 2e-10

In [ ]:
species = pd.DataFrame(['H3+','H2D+',"D2H+","D3+",'HD','H2'],columns=['name'])

reactions = [Reaction(species,[0,4],[1,5],k=1.43e-9),
             Reaction(species,[1,4],[2,5],k=1.33e-9),
             Reaction(species,[2,4],[3,5],k=1.05e-9),
             Reaction(species,[1,5],[0,4],k=2e-10),
             Reaction(species,[2,5],[1,4],k=2e-10),
             Reaction(species,[3,5],[2,4],k=2e-10)]

reactions = pd.DataFrame(reactions,columns=['reaction'])
print(reactions)

volume = 1

result = spi.solve_ivp(calc_derivative_2,(0,.15),y0=[932/volume,0,0,0,6.3e10,0],t_eval=df['time'],method='RK45',dense_output=True,args=[reactions])
fig,ax = plt.subplots(figsize=(10,8))
ax.set_prop_cycle(ucd_cycler)
t = np.linspace(0,150e-3,1000)
lines = ax.plot(t,(result.sol(t)[0:4]).T)
for l,n in zip(lines,species['name'][0:4]):
    l.set_label(n)
ax.scatter(df['time'],df['H3+']/volume,color='#000000')
ax.scatter(df['time'],df['H2D+']/volume,color='#ffbf00')
ax.scatter(df['time'],df['D2H+']/volume,color='#022851')
ax.scatter(df['time'],df['D3+']/volume,color='#c10230')
ax.set_xlim(0,150e-3)
ax.set_ylabel('Abundance (cm$^{-3}$)')
ax.set_yscale('log')
ax.legend()

It appears to make no difference! This is because in our model, the abundance of H$_2$ remains tiny. However, experimentally, the HD gas has a purity of only 97%. If we plug that in for the initial abundances, we can start to see something:

In [ ]:
species = pd.DataFrame(['H3+','H2D+',"D2H+","D3+",'HD','H2'],columns=['name'])

reactions = [Reaction(species,[0,4],[1,5],k=1.43e-9),
             Reaction(species,[1,4],[2,5],k=1.33e-9),
             Reaction(species,[2,4],[3,5],k=1.05e-9),
             Reaction(species,[1,5],[0,4],k=2e-10),
             Reaction(species,[2,5],[1,4],k=2e-10),
             Reaction(species,[3,5],[2,4],k=2e-10)]

reactions = pd.DataFrame(reactions,columns=['reaction'])
print(reactions)

volume = 1

result = spi.solve_ivp(calc_derivative_2,(0,.15),y0=[930/volume,0,0,0,0.97*6.3e10,0.03*6.3e10],t_eval=df['time'],method='RK45',dense_output=True,args=[reactions])

fig,ax = plt.subplots(figsize=(10,8))
ax.set_prop_cycle(ucd_cycler)
t = np.linspace(0,150e-3,1000)
lines = ax.plot(t,(result.sol(t)[0:4]*volume).T)
ax.scatter(df['time'],df['H3+'],color='#000000',label=r'H$_3^+$')
ax.scatter(df['time'],df['H2D+'],color='#ffbf00',label=r'H$_2$D$^+$')
ax.scatter(df['time'],df['D2H+'],color='#022851',label=r'D$_2$H$^+$')
ax.scatter(df['time'],df['D3+'],color='#c10230',label=r'D$_3^+$')
ax.set_ylim(0.1,2000)
ax.set_xlim(0,150e-3)
ax.legend(loc='lower left',bbox_to_anchor=(0.1,0.3))
ax.set_xlabel("Time (s)")
ax.set_ylabel('Ion count')
ax.set_yscale('log')

After some manual adjustment of the rate coefficients, we can obtain good agreement with the experimental data. It is theoretically possible to improve this with `least_squares`, but there are now 6 rate coefficients and an extra parameter for the percentage of H$_2$ that would need to be optimized as well, which makes the process slow. Also, some parameters have only a tiny effect on the data, so a lot of care has to be taken to ensure the optimization works well.

In [ ]:
species = pd.DataFrame(['H3+','H2D+',"D2H+","D3+",'HD','H2'],columns=['name'])

reactions = [Reaction(species,[0,4],[1,5],k=1.4e-9),
             Reaction(species,[1,4],[2,5],k=1.4e-9),
             Reaction(species,[2,4],[3,5],k=1.1e-9),
             Reaction(species,[1,5],[0,4],k=1e-10),
             Reaction(species,[2,5],[1,4],k=2e-10),
             Reaction(species,[3,5],[2,4],k=4e-10)]

reactions = pd.DataFrame(reactions,columns=['reaction'])
print(reactions)

volume = 1

result = spi.solve_ivp(calc_derivative_2,(0,.15),y0=[930/volume,0,0,0,0.97*6.3e10,0.03*6.3e10],t_eval=df['time'],method='RK45',dense_output=True,args=[reactions])

fig,ax = plt.subplots(figsize=(10,8))
ax.set_prop_cycle(ucd_cycler)
t = np.linspace(0,150e-3,1000)
lines = ax.plot(t,(result.sol(t)[0:4]*volume).T)
ax.scatter(df['time'],df['H3+'],color='#000000',label=r'H$_3^+$')
ax.scatter(df['time'],df['H2D+'],color='#ffbf00',label=r'H$_2$D$^+$')
ax.scatter(df['time'],df['D2H+'],color='#022851',label=r'D$_2$H$^+$')
ax.scatter(df['time'],df['D3+'],color='#c10230',label=r'D$_3^+$')
ax.set_ylim(0.1,2000)
ax.set_xlim(0,150e-3)
ax.legend(loc='lower left',bbox_to_anchor=(0.1,0.3))
ax.set_xlabel("Time (s)")
ax.set_ylabel('Ion count')
ax.set_yscale('log')

## Using Implicit Solvers

The implicit solvers that are good for stiff problems carry one additional complication: they require the Jacobian matrix in order to run efficiently. For a kinetics system with N molecules, the Jacobian matrix contains derivatives of the rates for each molecule with respect to every molecule:

$$ J_{ij} = \frac{\partial}{\partial [\text{X}]_j} \text{Rate}_i $$

For a reaction aA + bB --> cC + dD, we know the rates are:

$$ \frac{\text{d}[\text{A}]}{\text{d}t} = -ak[\text{A}]^a[\text{B}]^b, \quad  \frac{\text{d}[\text{B}]}{\text{d}t} = -bk[\text{A}]^a[\text{B}]^b, \quad \frac{\text{d}[\text{C}]}{\text{d}t} = ck[\text{A}]^a[\text{B}]^b, \quad \frac{\text{d}[\text{D}]}{\text{d}t} = -dk[\text{A}]^a[\text{B}]^b $$

Taking the rate for A as an example, the derivatives with respect to each molecule are:

$$ \frac{\partial}{\partial [\text{A}]} \text{Rate}_\text{A} = -aka[\text{A}]^{a-1}[\text{B}]^b, \quad \frac{\partial}{\partial [\text{B}]} \text{Rate}_\text{A} = -akb[\text{a}]^a[\text{B}]^{b-1}, \quad \frac{\partial}{\partial [\text{C}]} \text{Rate}_\text{A} = 0, \quad \frac{\partial}{\partial [\text{D}]} \text{Rate}_\text{A} = 0 $$

If we apply this to each rate, the Jacobian matrix for this reaction is:

$$ J = \begin{bmatrix} -aka[\text{A}]^{a-1}[\text{B}]^b & -akb[\text{A}]^a[\text{B}]^{b-1} & 0 & 0 \\ -bka[\text{A}]^{a-1}[\text{B}]^b & -bkb[\text{A}]^a[\text{B}]^{b-1} & 0 & 0 \\ cka[\text{A}]^{a-1}[\text{B}]^b & ckb[\text{A}]^a[\text{B}]^{b-1} & 0 & 0 \\ dka[\text{A}]^{a-1}[\text{B}]^b & dkb[\text{A}]^a[\text{B}]^{b-1} & 0 & 0\end{bmatrix} $$

Assuming our system contains two other molecules E and F, the total contribution to the Jacobian matrix for this one reaction would have 0s in all of the extra rows and columns because the rate of this reaction does not depend on the concentrations of E or F:

$$ J = \begin{bmatrix} -aka[\text{A}]^{a-1}[\text{B}]^b & -akb[\text{A}]^a[\text{B}]^{b-1} & 0 & 0 & 0 & 0 \\ -bka[\text{A}]^{a-1}[\text{B}]^b & -bkb[\text{A}]^a[\text{B}]^{b-1} & 0 & 0 & 0 & 0 \\ cka[\text{A}]^{a-1}[\text{B}]^b & ckb[\text{A}]^a[\text{B}]^{b-1} & 0 & 0 & 0 & 0 \\ dka[\text{A}]^{a-1}[\text{B}]^b & dkb[\text{A}]^a[\text{B}]^{b-1} & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 \end{bmatrix} $$

Then we can repeat the process for each reaction in the system, just adding to the appropriate elements of the Jacobian matrix. We can provide a function to calculate the Jacobian whose signature is `f(t,y,*args)` just like for `calc_derivative_2`.

In [ ]:
def calc_jacobian(t,y,rxns):
    J = np.zeros((y.size,y.size)) #create an empty NxN matrix, where N = number of molecules in the system
    for r in rxns['reaction']:
        #loop over reactants; each loop computes one column of the Jacobian matrix
        for i,(rc,ex) in enumerate(zip(r.reactants,r.rcounts)):
            out = np.zeros(y.size)
            
            #when we compute df/di, the power of reactant i is reduced by 1. So subtract 1 from the reactant counts at the ith position
            #However, we don't want to modify the reaction itself, so make a copy of rcounts
            ords = np.copy(r.rcounts)
            ords[i] -= 1
            
            #calculate the base rate = k * count * product (concentrations raised to correct powers)
            rate = r.k*ex*np.prod(np.power(y[r.reactants],ords))
            
            #rectants decrease by reactant count * base rate; products increase by product count * base rate
            out[r.reactants] -= r.rcounts*rate
            out[r.products] += r.pcounts*rate
            
            #add to the correct column of the Jacobian matrix for this reactant
            J[:,rc] += out
    
    return J

#play around with the reaction definition to ensure jacobian is calculated correctly, using formulas above
r = Reaction(species,[0,1,1],[2,2,3],2.)
y = np.asarray([10.,20.,30.,40.])
calc_jacobian(0,y,pd.DataFrame([r],columns=['reaction']))

For large systems of reactions, we can also define the jacobian's sparsity structure. This is a matrix that has 1s in the positions where the Jacobian may be nonzero, and 0s where the Jacobian is always 0. The algorithms in the `solve_ivp` function can use that information to speed up the calculations because it can reduce the number of calculations it needs to perform. When the reaction network is large, there may be only a few reactions linked to some molecules, and the rows/columns corresponding to that element may contain many 0s. The sparsity structure depends only on the reaction network, not the state of the system, so we can precalculate it before running `solve_ivp`. For completeness, we'll do it here.

In [ ]:
def compute_sparsity(species,rxns):
    out = np.zeros((species.size,species.size))
    for rxn in rxns['reaction']:
        for r in rxn.reactants:
            out[r,rxn.reactants] = 1
            for p in rxn.products:
                out[p,rxn.products] = 1
                out[r,p] = 1
                
    return out

species = pd.DataFrame(['H3+','H2D+',"D2H+","D3+",'HD','H2'],columns=['name'])

reactions = [Reaction(species,[0,4],[1,5],k=1.4e-9),
             Reaction(species,[1,4],[2,5],k=1.4e-9),
             Reaction(species,[2,4],[3,5],k=1.1e-9),
             Reaction(species,[1,5],[0,4],k=1e-10),
             Reaction(species,[2,5],[1,4],k=2e-10),
             Reaction(species,[3,5],[2,4],k=4e-10)]

reactions = pd.DataFrame(reactions,columns=['reaction'])
print(reactions)
compute_sparsity(species,reactions)

Of course, there are very few 0s in this matrix. H$_3^+$ is not directly linked to D$_2$H$^+$ or D$_3^+$, and H$_2$D$^+$ is not linked to D$_3^+$, but otherwise each molecule is connected by at least 1 reaction. Now that we have the Jacobian and the sparsity structure, we can use one of the implicit solvers. (Strictly speaking, it is possible to use an implicit solver without the Jacobian matrix, in which case the Jacobian can be estimated by finite differences. However, doing so is extremely slow and introduces additional error, so it should be avoided).

In [ ]:
species = pd.DataFrame(['H3+','H2D+',"D2H+","D3+",'HD','H2'],columns=['name'])

reactions = [Reaction(species,[0,4],[1,5],k=1.4e-9),
             Reaction(species,[1,4],[2,5],k=1.4e-9),
             Reaction(species,[2,4],[3,5],k=1.1e-9),
             Reaction(species,[1,5],[0,4],k=1e-10),
             Reaction(species,[2,5],[1,4],k=2e-10),
             Reaction(species,[3,5],[2,4],k=4e-10)]

reactions = pd.DataFrame(reactions,columns=['reaction'])
volume = 1.
print(reactions)
sparse = compute_sparsity(species,reactions)

result = spi.solve_ivp(calc_derivative_2,(0,.15),y0=[930/volume,0,0,0,0.97*6.3e10,0.03*6.3e10],t_eval=df['time'],
                       method='Radau',dense_output=True,args=[reactions],jac=calc_jacobian,jac_sparsity=sparse)

fig,ax = plt.subplots(figsize=(10,8))
ax.set_prop_cycle(ucd_cycler)
t = np.linspace(0,150e-3,1000)
lines = ax.plot(t,(result.sol(t)[0:4]*volume).T)
ax.scatter(df['time'],df['H3+'],color='#000000',label=r'H$_3^+$')
ax.scatter(df['time'],df['H2D+'],color='#ffbf00',label=r'H$_2$D$^+$')
ax.scatter(df['time'],df['D2H+'],color='#022851',label=r'D$_2$H$^+$')
ax.scatter(df['time'],df['D3+'],color='#c10230',label=r'D$_3^+$')
ax.set_ylim(0.1,2000)
ax.set_xlim(0,150e-3)
ax.legend(loc='lower left',bbox_to_anchor=(0.1,0.3),framealpha=0.)
ax.set_xlabel("Time (s)")
ax.set_ylabel('Ion count')
ax.set_yscale('log')